<a href="https://colab.research.google.com/github/rachit2005/UNET-/blob/main/video_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
import kagglehub

# Download latest version
# path = kagglehub.dataset_download("trainingdatapro/aggressive-behavior-video-classification")
path = kagglehub.dataset_download("pevogam/ucf101")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'ucf101' dataset.
Path to dataset files: /kaggle/input/ucf101


In [40]:
from torchvision.transforms.v2 import RandomHorizontalFlip, RandomVerticalFlip, Compose, Normalize, Resize, ToPILImage
from torch.utils.data import Dataset, ConcatDataset
from torchvision.transforms import ToTensor
from glob import glob
from PIL import Image
import shutil
import torch
import cv2
import os
import gc

In [41]:
import os

root = "/root/.cache/kagglehub/datasets/pevogam/ucf101/versions/1/UCF101/UCF-101"

print("Exists:", os.path.exists(root))
print("Top-level contents:")
print(os.listdir(root)[:20])  # show first 20 items

Exists: True
Top-level contents:
['BlowDryHair', 'RockClimbingIndoor', 'HulaHoop', 'PlayingFlute', 'SoccerJuggling', 'ApplyEyeMakeup', 'JumpRope', 'CliffDiving', 'BoxingPunchingBag', 'PoleVault', 'MilitaryParade', 'CleanAndJerk', 'BasketballDunk', 'Shotput', 'YoYo', 'Lunges', 'BandMarching', 'GolfSwing', 'PlayingDhol', 'CuttingInKitchen']


In [42]:
from cv2.gapi import video
dataset_url = "/root/.cache/kagglehub/datasets/pevogam/ucf101/versions/1/UCF101/UCF-101"

def extract_frames(video_path, output_dir, frame_rate=1):
  cap = cv2.VideoCapture(video_path)
  frame_num = 0
  frame_skip = 4

  while True:
    ret,frame = cap.read()
    if not ret:
      break
    if frame_num % frame_skip == 0:
      frame_path = os.path.join(output_dir , f'frame_{frame_num}.jpg')
      cv2.imwrite(frame_path, frame)
    frame_num += 1

  cap.release()


def get_video(dataset_url):
    video_paths = glob(os.path.join(dataset_url, "**", "*.avi"), recursive=True)
    return video_paths


In [43]:
videos = get_video(dataset_url)

print(videos)
print(len(videos))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

['/root/.cache/kagglehub/datasets/pevogam/ucf101/versions/1/UCF101/UCF-101/BlowDryHair/v_BlowDryHair_g03_c05.avi', '/root/.cache/kagglehub/datasets/pevogam/ucf101/versions/1/UCF101/UCF-101/BlowDryHair/v_BlowDryHair_g09_c01.avi', '/root/.cache/kagglehub/datasets/pevogam/ucf101/versions/1/UCF101/UCF-101/BlowDryHair/v_BlowDryHair_g17_c02.avi', '/root/.cache/kagglehub/datasets/pevogam/ucf101/versions/1/UCF101/UCF-101/BlowDryHair/v_BlowDryHair_g17_c04.avi', '/root/.cache/kagglehub/datasets/pevogam/ucf101/versions/1/UCF101/UCF-101/BlowDryHair/v_BlowDryHair_g10_c01.avi', '/root/.cache/kagglehub/datasets/pevogam/ucf101/versions/1/UCF101/UCF-101/BlowDryHair/v_BlowDryHair_g25_c01.avi', '/root/.cache/kagglehub/datasets/pevogam/ucf101/versions/1/UCF101/UCF-101/BlowDryHair/v_BlowDryHair_g01_c04.avi', '/root/.cache/kagglehub/datasets/pevogam/ucf101/versions/1/UCF101/UCF-101/BlowDryHair/v_BlowDryHair_g14_c05.avi', '/root/.cache/kagglehub/datasets/pevogam/ucf101/versions/1/UCF101/UCF-101/BlowDryHair/v

In [44]:
import os
import cv2
import torch
from torch.utils.data import Dataset
from PIL import Image

class VideoDataset(Dataset):
    def __init__(self, root_dir, video_paths, class_to_idx, transforms=None, frame_skip=4, max_frames=32):
        self.root_dir = root_dir
        self.video_paths = video_paths
        self.class_to_idx = class_to_idx
        self.transforms = transforms
        self.frame_skip = frame_skip
        self.max_frames = max_frames

    def __len__(self):
        return len(self.video_paths)

    def __getitem__(self, index):
        video_path = self.video_paths[index]

        #  Get class name from parent folder
        class_name = os.path.basename(os.path.dirname(video_path))
        label = self.class_to_idx[class_name]
        label = torch.tensor(label, dtype=torch.long)

        cap = cv2.VideoCapture(video_path)

        frames = []
        frame_idx = 0
        saved = 0

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            if frame_idx % self.frame_skip == 0:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                img = Image.fromarray(frame)

                if self.transforms:
                    img = self.transforms(img)  # (C, H, W)

                frames.append(img)
                saved += 1

                if saved >= self.max_frames:
                    break

            frame_idx += 1

        cap.release()

        # If video is too short, pad with zeros
        if len(frames) == 0:
            # Fallback dummy
            video = torch.zeros(3, 1, 112, 112)
            return video, label

        # Stack → (T, C, H, W)
        video_tensor = torch.stack(frames, dim=0)

        # Permute → (C, T, H, W)
        video = video_tensor.permute(1, 0, 2, 3)

        C,T,H,W = video.shape
        target_T = self.max_frames

        if T<target_T:
            pad = torch.zeros(C,target_T-T,H,W , device=device)
            video = torch.cat([video,pad],dim=1)
        else:
          video = video[:,:target_T,:,:]

        return video, label


In [45]:
# classes bna rhe hai

classes = sorted(os.listdir(dataset_url))
class_to_idx = {cls: i for i, cls in enumerate(classes)}

print("Number of classes:", len(classes))
print("First 10 classes:", classes[:10])

Number of classes: 101
First 10 classes: ['ApplyEyeMakeup', 'ApplyLipstick', 'Archery', 'BabyCrawling', 'BalanceBeam', 'BandMarching', 'BaseballPitch', 'Basketball', 'BasketballDunk', 'BenchPress']


In [46]:
effects = Compose([
    ToPILImage(),
    RandomHorizontalFlip(),
    RandomVerticalFlip(),
    Resize(size = (112, 112), antialias = True),
    ToTensor(),
    Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
])

from glob import glob

dataset_url = "/root/.cache/kagglehub/datasets/pevogam/ucf101/versions/1/UCF101/UCF-101"

# Get class names
classes = sorted(os.listdir(dataset_url))
class_to_idx = {cls: i for i, cls in enumerate(classes)}

# Get all videos
video_paths = glob(os.path.join(dataset_url, "**", "*.avi"), recursive=True)

print("Classes:", len(classes))
print("Videos:", len(video_paths))

dataset = VideoDataset(
    root_dir=dataset_url,
    video_paths=video_paths,
    class_to_idx=class_to_idx,
    transforms=effects,
    frame_skip=4,      # take 1 frame every 4
    max_frames=32      # clip length
)

print("Dataset size:", len(dataset))

Classes: 101
Videos: 13320
Dataset size: 13320


In [47]:
import os

for root, dirs, files in os.walk(path):
    print(f"Directory: {root}")
    for file in files:
        print(f"  - {file}")

Streaming output truncated to the last 5000 lines.
  - v_JumpingJack_g17_c03.avi
  - v_JumpingJack_g25_c04.avi
  - v_JumpingJack_g25_c06.avi
  - v_JumpingJack_g07_c04.avi
  - v_JumpingJack_g12_c03.avi
  - v_JumpingJack_g02_c03.avi
  - v_JumpingJack_g22_c03.avi
  - v_JumpingJack_g05_c06.avi
  - v_JumpingJack_g02_c04.avi
  - v_JumpingJack_g13_c07.avi
  - v_JumpingJack_g06_c06.avi
  - v_JumpingJack_g17_c01.avi
  - v_JumpingJack_g21_c03.avi
  - v_JumpingJack_g05_c04.avi
  - v_JumpingJack_g25_c05.avi
  - v_JumpingJack_g23_c02.avi
  - v_JumpingJack_g20_c03.avi
  - v_JumpingJack_g20_c01.avi
  - v_JumpingJack_g24_c01.avi
  - v_JumpingJack_g10_c06.avi
  - v_JumpingJack_g16_c01.avi
  - v_JumpingJack_g09_c02.avi
  - v_JumpingJack_g01_c05.avi
  - v_JumpingJack_g12_c02.avi
  - v_JumpingJack_g18_c01.avi
  - v_JumpingJack_g10_c05.avi
  - v_JumpingJack_g01_c03.avi
  - v_JumpingJack_g10_c01.avi
  - v_JumpingJack_g12_c01.avi
  - v_JumpingJack_g03_c02.avi
  - v_JumpingJack_g23_c01.avi
  - v_JumpingJack_g

In [48]:
from torch.utils.data import DataLoader, random_split
print(len(dataset))

train_size = int(0.8*len(dataset))
eval_size = len(dataset) - train_size

train_dataset , test_dataset = random_split(dataset , [train_size, eval_size])

print(len(train_dataset))
print(len(test_dataset))

batch_size = 2

train_dataloader = DataLoader(train_dataset , batch_size = batch_size , shuffle = True , num_workers=0)
test_dataloder = DataLoader(test_dataset , batch_size , shuffle = True)

print(next(iter(train_dataloader)))

print(len(train_dataloader.dataset))

13320
10656
2664
[tensor([[[[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
           [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
           [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
           ...,
           [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
           [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
           [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],

          [[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
           [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
           [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
           ...,
           [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
           [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
           [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],

          [[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179

In [49]:
! pip install snntorch

In [50]:
import torch
import torch.nn as nn
import torch.ao.quantization as quantization
import snntorch as snn
from snntorch import surrogate


class Conv3dBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()

        self.quant = quantization.QuantStub()
        self.dequant = quantization.DeQuantStub()

        self.conv = nn.Conv3d(in_channels, out_channels, 3, padding=1, bias=False)
        self.bn = nn.BatchNorm3d(out_channels)
        self.relu = nn.ReLU(inplace=True)

        self.lif = snn.Leaky(
            beta=0.95,
            spike_grad=surrogate.fast_sigmoid(),
            init_hidden=True
        )

    def forward(self, x):
        x = self.quant(x)
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.dequant(x)

        spk = self.lif(x)
        return spk



class UNET_VIDEO_CLASSIFICATION(nn.Module):
    def __init__(self, num_classes=101, in_channels=3):
        super(UNET_VIDEO_CLASSIFICATION, self).__init__()

        # --- Quantization Stubs ---
        # QuantStub converts floating point tensors to quantized tensors
        self.quant = quantization.QuantStub()
        # DeQuantStub converts quantized tensors back to floating point
        self.dequant = quantization.DeQuantStub()

        # --- Contracting Path (Encoder) ---
        # Level 1
        self.enc1 = Conv3dBlock(in_channels, 32)
        self.pool1 = nn.MaxPool3d(kernel_size=(1,2,2), stride=(1,2,2))


        # Level 2
        self.enc2 = Conv3dBlock(32, 64)
        self.pool2 = nn.MaxPool3d(kernel_size=(1,2,2), stride=(1,2,2))


        # Level 3
        self.enc3 = Conv3dBlock(64, 128)
        self.pool3 = nn.MaxPool3d(kernel_size=(1,2,2), stride=(1,2,2))


        # Level 4 (Bottleneck)
        self.bottleneck = Conv3dBlock(128, 256)

        # --- Classification Head ---
        # Replaces the U-Net Decoder for classification tasks
        self.global_pool = nn.AdaptiveAvgPool3d((1, 1, 1))
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(256, num_classes)

    def forward(self, x):
      # x shape: [B, C, T, H, W]

      self.reset_snn()

      time_steps = x.shape[2]
      spike_sum = torch.zeros(x.shape[0], self.fc.out_features, device=x.device)

      for t in range(time_steps):
          xt = x[:, :, t:t+1, :, :]

          # Quantize
          xt = self.quant(xt)

          # Encoder
          xt = self.pool1(self.enc1(xt))
          xt = self.pool2(self.enc2(xt))
          xt = self.pool3(self.enc3(xt))

          # Bottleneck
          xt = self.bottleneck(xt)

          # Global Pool
          xt = self.global_pool(xt)
          xt = torch.flatten(xt, 1)
          xt = self.dropout(xt)
          xt = self.fc(xt)

          # Accumulate spikes
          spike_sum += xt

      # Average over time
      out = spike_sum / time_steps

      # Dequantize
      out = self.dequant(out)
      return out

    def reset_snn(self):
      for m in self.modules():
          if isinstance(m, snn.Leaky):
              m.reset_hidden()

    def fuse_model(self):
        """
        Fuses Conv+BN+ReLU layers to save memory and improve speed.
        Required for effective Quantization Aware Training.
        """
        # Fuse the sub-blocks (Conv + BN + ReLU) by specifying full paths from the model
        torch.ao.quantization.fuse_modules(
            self, ['enc1.conv', 'enc1.bn', 'enc1.relu'], inplace=True
        )
        torch.ao.quantization.fuse_modules(
            self, ['enc2.conv', 'enc2.bn', 'enc2.relu'], inplace=True
        )
        torch.ao.quantization.fuse_modules(
            self, ['enc3.conv', 'enc3.bn', 'enc3.relu'], inplace=True
        )
        torch.ao.quantization.fuse_modules(
            self, ['bottleneck.conv', 'bottleneck.bn', 'bottleneck.relu'], inplace=True
        )

In [51]:
model = UNET_VIDEO_CLASSIFICATION(num_classes=101)

print(model)
print(len(dataset))

UNET_VIDEO_CLASSIFICATION(
  (quant): QuantStub()
  (dequant): DeQuantStub()
  (enc1): Conv3dBlock(
    (quant): QuantStub()
    (dequant): DeQuantStub()
    (conv): Conv3d(3, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
    (bn): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (lif): Leaky()
  )
  (pool1): MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (enc2): Conv3dBlock(
    (quant): QuantStub()
    (dequant): DeQuantStub()
    (conv): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
    (bn): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (lif): Leaky()
  )
  (pool2): MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (enc3): Conv3dBlock(
    (quant): QuantStub()
    (dequant): DeQuantStub()
   

## Quantization aware training

In [52]:
import torch
import torch.nn as nn
import torch.ao.quantization as quantization

BATCH_SIZE = 2
EPOCHS = 5
LR = 3e-4

loss_fn = nn.CrossEntropyLoss()

def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # ---- Fuse model (must be in eval) ----
    model.eval()
    model.fuse_model()
    model.train()

    model.to(device)

    backend = "fbgemm"
    model.qconfig = quantization.get_default_qat_qconfig(backend)

    model.fc.qconfig = None
    model.dropout.qconfig = None
    model.global_pool.qconfig = None

    import snntorch as snn
    for m in model.modules():
        if isinstance(m, snn.Leaky):
            m.qconfig = None

    quantization.prepare_qat(model, inplace=True)

    optimizer = torch.optim.Adam(model.parameters(), lr=LR)

    print("Starting Quantization Aware Training")

    for epoch in range(EPOCHS):
        model.train()
        total_loss = 0.0
        correct = 0
        total = 0

        for videos, labels in train_dataloader:
            videos = videos.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            output = model(videos)
            loss = loss_fn(output, labels)

            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            _, predicted = torch.max(output, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_loss = total_loss / len(train_dataloader)
        train_accuracy = 100.0 * correct / total

        print(f"Epoch [{epoch+1}/{EPOCHS}] | Loss: {train_loss:.4f} | Accuracy: {train_accuracy:.2f}%")


In [ ]:
main()

/tmp/ipython-input-2795508726.py:33: DeprecationWarning: torch.ao.quantization is deprecated and will be removed in 2.10. 
For migrations of users: 
1. Eager mode quantization (torch.ao.quantization.quantize, torch.ao.quantization.quantize_dynamic), please migrate to use torchao eager mode quantize_ API instead 
2. FX graph mode quantization (torch.ao.quantization.quantize_fx.prepare_fx,torch.ao.quantization.quantize_fx.convert_fx, please migrate to use torchao pt2e quantization API instead (prepare_pt2e, convert_pt2e) 
3. pt2e quantization has been migrated to torchao (https://github.com/pytorch/ao/tree/main/torchao/quantization/pt2e) 
see https://github.com/pytorch/ao/issues/2259 for more details
  quantization.prepare_qat(model, inplace=True)
/usr/local/lib/python3.12/dist-packages/torch/ao/quantization/observer.py:246: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorc

Starting Quantization Aware Training


In [ ]:
model.eval()
model.to('cpu')

# Perform conversion in-place to avoid deepcopy issues
quantization.convert(model , inplace=True)
quantized_model = model # Assign the in-place converted model

print("conversion succesfull")

torch.save(quantized_model.state_dict(), 'quantized_model.pth')
torch.save(model.state_dict() , "model.pth")